<a href="https://colab.research.google.com/github/rikanga/Easy-Numpy/blob/openclassroom-datascience-tools/analyse2_openclassroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np, pandas as pd, re

In [2]:
data = pd.read_csv('https://s3.eu-west-1.amazonaws.com/course.oc-static.com/courses/7410486/personnes.csv')

In [3]:
data.head()

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m


In [4]:
data.describe()

,prenom,email,date_naissance,pays,taille
count,7,7,6,6,7
unique,7,6,6,5,7
top,Leila,samuel_329@example.com,23/01/1990,France,1.49m
freq,1,2,1,2,1


In [5]:
data

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
5,Hanna,hanna2019@supermail.eu,01/01/1970,24,3.45m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


### **Valeurs maquantes**

In [6]:
data.isnull()

,prenom,email,date_naissance,pays,taille
0,False,False,False,False,False
1,False,False,False,True,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
5,False,False,False,False,False
6,False,False,True,False,False


**Compter le nombre des fois où la valeur est manquante**

In [7]:
data.isnull().sum()

prenom            0
email             0
date_naissance    1
pays              1
taille            0
dtype: int64

### **Les doublons**

In [8]:
data.loc[data['email'].duplicated(keep=False)]

,prenom,email,date_naissance,pays,taille
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


In [9]:
data.loc[data['email'].duplicated(keep=False), :]

,prenom,email,date_naissance,pays,taille
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


In [10]:
data.loc[data.email.duplicated(), :]

,prenom,email,date_naissance,pays,taille
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


### **Traiter les erreurs**

**Traiter les pays**

In [11]:
VALID_COUNTRIES = ['France', "Côte d'ivoire", 'Madagascar', 'Bénin', 'Allemagne',
                   'USA']

In [12]:
data['pays'].isin(VALID_COUNTRIES)

0     True
1    False
2     True
3     True
4     True
5    False
6     True
Name: pays, dtype: bool

In [13]:
~data['pays'].isin(VALID_COUNTRIES)

0    False
1     True
2    False
3    False
4    False
5     True
6    False
Name: pays, dtype: bool

In [14]:
mask = ~data['pays'].isin(VALID_COUNTRIES)

In [15]:
# Indivu avec des pays non valides
data.loc[mask]

,prenom,email,date_naissance,pays,taille
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
5,Hanna,hanna2019@supermail.eu,01/01/1970,24,3.45m


**Remplacer les pays par des valeurs nulles**

In [16]:
# On replace les individus avec le pays non valide par une valeur nulle
data.loc[mask, 'pays'] = np.NaN

In [17]:
data

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
5,Hanna,hanna2019@supermail.eu,01/01/1970,NaN,3.45m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


**Traiter les adresses mails**

In [18]:
# expand=True, --> met les différentes sérpations sur plusieurs colonnes
# n=1, --> Le nombre des fois que l'on doit faire un split
data['email'].str.split(',', expand=True, n=1)

,0,1
0,leila@example.com,None
1,samuel_329@example.com,None
2,choupipoune@supermail.eu,None
3,marco23@example.com,mc23@supermail.eu
4,helloworld@supermail.eu,None
5,hanna2019@supermail.eu,None
6,samuel_329@example.com,None


In [19]:
# On sélectionne juste la prémière colonne
data['email'].str.split(',', expand=True, n=1)[0]

0           leila@example.com
1      samuel_329@example.com
2    choupipoune@supermail.eu
3         marco23@example.com
4     helloworld@supermail.eu
5      hanna2019@supermail.eu
6      samuel_329@example.com
Name: 0, dtype: object

In [20]:
# On replace la colonne email par les nouvelles valeurs
data['email'] = data['email'].str.split(',', expand=True, n=1)[0]

In [21]:
# Nouveau dataset avec un email sur chaque colonne 'email'
data

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,marco23@example.com,10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
5,Hanna,hanna2019@supermail.eu,01/01/1970,NaN,3.45m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


**Traiter les tailles**

In [22]:
data['taille']

0    1.49m
1    1.67m
2    153cm
3    1.65m
4    1.34m
5    3.45m
6    1.45m
Name: taille, dtype: object

In [23]:
# On prend toutes les valeurs sauf la dernière qui est une lettre
data['taille'].str[:-1]

0    1.49
1    1.67
2    153c
3    1.65
4    1.34
5    3.45
6    1.45
Name: taille, dtype: object

In [24]:
# On remplace la colonne par des nouvelles données
data['taille'] = data['taille'].str[:-1]
data

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153c
3,Marc,marco23@example.com,10/02/1978,France,1.65
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34
5,Hanna,hanna2019@supermail.eu,01/01/1970,NaN,3.45
6,samuël,samuel_329@example.com,NaN,Bénin,1.45


In [25]:
# Convertir la taille en entier
data['taille'] = pd.to_numeric(data['taille'], errors='coerce')

In [26]:
data

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,NaN
3,Marc,marco23@example.com,10/02/1978,France,1.65
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34
5,Hanna,hanna2019@supermail.eu,01/01/1970,NaN,3.45
6,samuël,samuel_329@example.com,NaN,Bénin,1.45


**Remplacer les valeurs maquantes**

In [27]:
# On remplace la valeur par la moyenne de de toutes les valeurs dans ce cas
data.loc[data['taille'].isnull(), 'taille'] = data['taille'].mean()

In [28]:
data

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.490000
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.670000
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,1.841667
3,Marc,marco23@example.com,10/02/1978,France,1.650000
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.340000
5,Hanna,hanna2019@supermail.eu,01/01/1970,NaN,3.450000
6,samuël,samuel_329@example.com,NaN,Bénin,1.450000
